### RunnablePassthrough()

In [1]:
# 필요한 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 모델 초기화
model = ChatOpenAI(model="gpt-4.1-nano")
prompt = ChatPromptTemplate.from_template(
    "{country}의 여행 할 만한 여행지를 3곳만 추천해줘."
)
chain1 = {"country": RunnablePassthrough()} | prompt | model | StrOutputParser()

print(chain1.invoke({"topic": "일본"}))

일본에서 여행하기 좋은 추천 여행지 3곳을 소개해 드릴게요:

1. 도쿄 (Tokyo)  
도쿄는 일본의 수도로 현대적인 도시와 전통이 조화를 이루는 곳입니다. 시부야의 번화한 거리, 신주쿠의 야경, 하라주쿠의 패션 거리, 아사쿠사의 센소지 사원 등 다양한 명소를 즐기실 수 있어요. 또한, 디즈니랜드 도쿄와 오다이바 같은 엔터테인먼트 시설도 인기입니다.

2. 교토 (Kyoto)  
전통과 역사가 깃든 도시 교토는 일본의 옛 수도로, 아름다운 겐로쿠엔 정원, 기온 거리, 금각사와 은각사 등 많은 사원과 찻집이 있습니다. 평온한 분위기 속에서 일본 전통 문화를 깊이 체험할 수 있는 곳이에요.

3. 오사카 (Osaka)  
맛집과 활기찬 거리로 유명한 오사카는 독특한 도시 분위기를 느낄 수 있습니다. 도톤보리 지역의 길거리 음식, 오사카성, 유니버설 스튜디오 재팬 등 즐길 거리가 풍부합니다. 특히 미식가 여행지로도 유명하니 다양한 일본 요리를 맛보기에 딱 좋아요.

이 세 곳은 각각의 매력이 넘쳐서 일본 여행을 더욱 특별하게 만들어 줄 거예요!


In [2]:
prompt = ChatPromptTemplate.from_template(
    "김포공항에서 {country}을 직항으로 갈 수 있는 공항을 3개만 추천해줘"
)
chain2 = {"country": RunnablePassthrough()} | prompt | model | StrOutputParser()
print(chain2.invoke({"country": "일본"}))

김포공항에서 일본으로 직항 항공편을 이용할 수 있는 대표적인 공항 세 곳을 추천드립니다:

1. 도쿄 하네다 공항 (Tokyo Haneda Airport, HND)
2. 오사카 간사이 공항 (Osaka Kansai International Airport, KIX)
3. 나리타 공항 (Tokyo Narita Airport, NRT)

이 공항들은 김포공항과 직항으로 연결되어 있어 편리하게 일본 내 주요 도시로 이동하실 수 있습니다. 비행 일정과 항공사에 따라 다소 차이가 있을 수 있으니, 예약 전에 최신 정보를 확인하시기 바랍니다.


### RunnablePassthrough() + RunnableParallel()

In [3]:
# 필요한 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
import json

# 모델 초기화 (gpt-4o-mini는 빠르고 저렴하여 테스트에 용이합니다)
model = ChatOpenAI(model="gpt-4o-mini")

# 1. 여행지 추천 체인
prompt1 = ChatPromptTemplate.from_template(
    "{country}의 여행할 만한 여행지를 3곳만 추천하고, 각 장소의 특징을 한 줄로 설명해줘."
)
chain1 = (
    # 입력이 문자열일 때, {"country": 입력값} 형태로 변환해 프롬프트에 전달
    {"country": RunnablePassthrough()}
    | prompt1
    | model
    | StrOutputParser()
)

# 2. 직항 공항 추천 체인
prompt2 = ChatPromptTemplate.from_template(
    "대한민국 김포공항에서 {country}(으)로 갈 수 있는 직항 공항을 3개만 알려줘."
)
chain2 = {"country": RunnablePassthrough()} | prompt2 | model | StrOutputParser()

# 3. RunnableParallel을 사용하여 두 체인 통합
# 'travel_info'와 'flight_info'라는 키로 각 체인의 결과를 받음
combined_chain = RunnableParallel(travel_info=chain1, flight_info=chain2)

combined_chain

{
  travel_info: {
                 country: RunnablePassthrough()
               }
               | ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 여행할 만한 여행지를 3곳만 추천하고, 각 장소의 특징을 한 줄로 설명해줘.'), additional_kwargs={})])
               | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001E8A3EE35D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001E8CA34DC90>, root_client=<openai.OpenAI object at 0x000001E8A0CDAA50>, root_async_client=<openai.AsyncOpenAI object at 0x000001E8CA40A710>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
               | StrOutputParser(),
  flight_info: {
                 country: RunnablePassthrough()
               }
               | ChatPromptTemplate

In [4]:
# 4. 통합된 체인 실행
# "일본"이라는 하나의 입력값이 chain1과 chain2에 동시에 전달됨
result = combined_chain.invoke("일본")

print(result["travel_info"])  # 여행지 추천 결과
print("=" * 70)
print(result["flight_info"])  # 직항 공항 추천 결과

1. **도쿄**: 현대적인 도시 풍경과 전통이 어우러진 곳으로, 쇼핑, 음식, 문화 체험이 풍부하다.

2. **교토**: 역사적인 사원과 아름다운 정원이 가득한 도시로, 일본의 전통 문화와 유산을 경험할 수 있다.

3. **홋카이도**: 자연의 아름다움이 돋보이는 지역으로, 스키와 온천, 신선한 해산물을 즐길 수 있는 여행지이다.
대한민국 김포공항에서 직항으로 갈 수 있는 일본의 공항은 다음 세 곳입니다:

1. 도쿄 하네다 공항 (Haneda Airport, HND)
2. 오사카 간사이 공항 (Kansai International Airport, KIX)
3. 후쿠오카 공항 (Fukuoka Airport, FUK)

이 외에도 다른 일본 도시로 가는 직항 노선이 있을 수 있으니, 항공사와 스케줄을 확인하는 것이 좋습니다.


### RunnableLambda

In [5]:
# 필요한 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableParallel,
    RunnableLambda,
)
from langchain_core.output_parsers import StrOutputParser
import json
import random  # random 라이브러리 추가

# --- (이전 코드와 동일한 부분) ---

# 모델 초기화 (gpt-4o-mini는 빠르고 저렴하여 테스트에 용이합니다)
model = ChatOpenAI(model="gpt-4o-mini")

# 1. 여행지 추천 체인
prompt1 = ChatPromptTemplate.from_template(
    "{country}의 여행할 만한 여행지를 3곳만 추천하고, 각 장소의 특징을 한 줄로 설명해줘."
)
chain1 = {"country": RunnablePassthrough()} | prompt1 | model | StrOutputParser()

# 2. 직항 공항 추천 체인
prompt2 = ChatPromptTemplate.from_template(
    "대한민국 김포공항에서 {country}(으)로 갈 수 있는 직항 공항을 3개만 알려줘."
)
chain2 = {"country": RunnablePassthrough()} | prompt2 | model | StrOutputParser()

# 3. RunnableParallel을 사용하여 두 체인 통합
combined_chain = RunnableParallel(travel_info=chain1, flight_info=chain2)


# 4. 국가를 랜덤으로 선택하는 함수 정의
def choice_country(_):  # LangChain 체인과 연결하기 위해 입력 인자를 받도록 정의합니다.
    """주어진 리스트에서 국가를 하나 랜덤으로 선택합니다."""
    countries = ["미국", "영국", "프랑스", "일본", "스위스"]
    selected_country = random.choice(countries)

    # 어떤 나라가 선택되었는지 확인하기 위해 출력
    print(f"🌍 랜덤 선택된 나라: {selected_country}")

    return selected_country # 랜덤으로 선택된 나라를 출력


# 5. RunnableLambda로 함수를 체인에 통합하고 최종 체인 완성
# [나라 선택 함수] -> [기존의 병렬 체인] 순서로 실행
final_chain = RunnableLambda(choice_country) | combined_chain

# 6. 최종 체인 실행
# 입력값이 필요 없으므로 None을 전달 (또는 비워둠)
result = final_chain.invoke(None)

# 7. 결과 출력
print("\n--- 최종 결과 ---")
print(json.dumps(result, indent=2, ensure_ascii=False))

🌍 랜덤 선택된 나라: 일본

--- 최종 결과 ---
{
  "travel_info": "1. **도쿄**: 현대와 전통이 조화를 이루는 대도시로, 다양한 문화와 맛있는 음식, 쇼핑의 중심지입니다.\n\n2. **교토**: 일본의 전통적인 아름다움을 간직한 도시로, 역사적인 사찰과 정원이 가득하여 고풍스러운 매력을 제공합니다.\n\n3. **홋카이도**: 눈부신 자연경관과 다양한 야외 활동을 즐길 수 있는 지역으로, 특히 겨울에는 스키와 온천이 유명합니다.",
  "flight_info": "대한민국 김포공항에서 일본으로 갈 수 있는 직항 공항은 다음과 같습니다:\n\n1. 도쿄 하네다 공항 (Tokyo Haneda Airport)\n2. 오사카 간사이 공항 (Kansai International Airport)\n3. 후쿠오카 공항 (Fukuoka Airport)\n\n이 외에도 일본의 여러 도시로 직항 노선이 있을 수 있으니, 항공편 정보 확인을 권장합니다."
}


### itemgetter()
 - 일반 파이썬 함수임

In [6]:
from operator import itemgetter

students_dict = [
    {"name": "홍길동", "age": 25, "score": 88},
    {"name": "이순신", "age": 32, "score": 95},
    {"name": "김유신", "age": 22, "score": 76},
]

# 딕셔너리의 'age' 키로 정렬
sorted_by_age = sorted(students_dict, key=itemgetter("age"))
print(f"\n딕셔너리 나이 정렬: {sorted_by_age}")

# 여러 항목(이름과 점수)을 한 번에 가져오기
get_name_and_score = itemgetter("name", "score")
for student in students_dict:
    print(f"가져온 항목: {get_name_and_score(student)}")


딕셔너리 나이 정렬: [{'name': '김유신', 'age': 22, 'score': 76}, {'name': '홍길동', 'age': 25, 'score': 88}, {'name': '이순신', 'age': 32, 'score': 95}]
가져온 항목: ('홍길동', 88)
가져온 항목: ('이순신', 95)
가져온 항목: ('김유신', 76)
